In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openbb import obb
import warnings
warnings.filterwarnings("ignore")
obb.user.preferences.output_type = "dataframe"

from scipy.stats import percentileofscore
import os
import math

# Calculate bullish percentage of 1,500 stocks

The constituents of the S&P 500, the S&P Midcap 400, and the S&P Smallcap 600

## Before starting, delete, move, or rename the files written in the last execution

point_and_figure_daily_trend_sp500.csv, point_and_figure_daily_trend_midcap400.csv, and point_and_figure_daily_trend_smallcap600.csv

## Identify trend of each stock's point and figure chart

i.e., is the current column X's or O's

### Set point and figure parameters for stocks

In [2]:
reversal_threshold = 3   # This must be a positive integer

FILEPATH = 'c:/Users/steve/Downloads/'  # For writing data to disk

start_date_string = (pd.Timestamp.today() - pd.Timedelta(days=1461)).strftime("%Y-%m-%d") # 4 years ago
end_date_string = (pd.Timestamp.today() + pd.DateOffset(1)).strftime("%Y-%m-%d")

### Function to determine the point and figure chart columns for a stock

In [3]:
def calc_point_and_figure(df):
    # ## Calculate point and figure history

    # ### Convert prices to logs

    # For percentage logic, convert the prices to logs based on the percentage
    df['log_high'] = np.log(df['high']) / np.log(1 + (box_percentage / 100))
    df['log_low'] = np.log(df['low']) / np.log(1 + (box_percentage / 100))
    df['log_close'] = np.log(df['close']) / np.log(1 + (box_percentage / 100))

    # Add more columns for determining point and figure
    df['trend'] = None
    df['last_box'] = None
    df['last_box_log'] = 0   # 20250520 SE - this needs to be an integer type
    df['reversal'] = False
    df['reversal_box'] = None
    df['new_box'] = None
    df['column_height'] = 0

    # Typically there is an initial lookback period in which it can't be known whether the point and figure trend is up (X boxes) or down (O boxes).
    # Allow for a provisional period at the beginning before any trend is known
    df['left_truncate'] = False

    # ### Initialize lists for point and figure chart
    extremes_log = []
    columns_log = []

    # ### Do point and figure calculations for first bar
    # Set a provisional initial box value
    initial_box = round(np.log(df['open'].iloc[0]) / np.log(1 + (box_percentage / 100)), 0).astype(int)

    # Use this initial value as the starting point
    extremes_log.append(initial_box)

    # Process the first row of data
    if df['log_high'].iloc[0] >= initial_box + reversal_threshold:
        df['trend'].iloc[0] = 'up'
        df['last_box_log'].iloc[0] = math.floor(df['log_high'].iloc[0])  # 20250520 SE round down even if log is negative
        df['last_box'].iloc[0] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[0]
        df['reversal'].iloc[0] = True
        df['reversal_box'].iloc[0] = (1 + (box_percentage / 100)) ** (initial_box + reversal_threshold)
        df['new_box'].iloc[0] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[0]
        df['column_height'].iloc[0] = df['last_box_log'].iloc[0] - initial_box
        columns_log.append(initial_box + 1)

    elif df['log_low'].iloc[0] <= initial_box - reversal_threshold:
        df['trend'].iloc[0] = 'down'
        df['last_box_log'].iloc[0] = math.ceil(df['log_low'].iloc[0])  # 20250520 SE round up even if log is negative
        df['last_box'].iloc[0] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[0]
        df['reversal'].iloc[0] = True
        df['reversal_box'].iloc[0] = (1 + (box_percentage / 100)) ** (initial_box - reversal_threshold)
        df['new_box'].iloc[0] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[0]
        df['column_height'].iloc[0] = initial_box - df['last_box_log'].iloc[0]
        columns_log.append(initial_box - 1)

    else:
        df['left_truncate'].iloc[0] = True
        df['last_box'].iloc[0] = (1 + (box_percentage / 100)) ** initial_box
        df['last_box_log'].iloc[0] = initial_box

    # ### Do point and figure calculations for all the other bars
    # 20250514 SE: Contrary to the way I initially coded this, the Dorsey book says to check for a continuation first, then reversal.

    for i in range(1, df.shape[0]):
        # If there is an uptrend
        if df['trend'].iloc[i-1] == 'up':
            # Check for new box(es) to the upside
            if df['log_high'].iloc[i] >= (df['last_box_log'].iloc[i-1] + 1):
                df['trend'].iloc[i] = df['trend'].iloc[i-1]
                df['last_box_log'].iloc[i] = math.floor(df['log_high'].iloc[i])  # 20250520 SE round down even if log is negative
                df['last_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['new_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['column_height'].iloc[i] = df['last_box_log'].iloc[i] - extremes_log[-1]

            # Check for reversal
            elif df['log_low'].iloc[i] <= df['last_box_log'].iloc[i-1] - reversal_threshold:
                df['trend'].iloc[i] = 'down'
                df['last_box_log'].iloc[i] = math.ceil(df['log_low'].iloc[i])  # 20250520 SE round up even if log is negative
                df['last_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['reversal'].iloc[i] = True
                df['reversal_box'].iloc[i] = (1 + (box_percentage / 100)) ** (df['last_box_log'].iloc[i-1] - reversal_threshold)
                df['new_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['column_height'].iloc[i] = df['last_box_log'].iloc[i-1] - df['last_box_log'].iloc[i]
                # Record the top box of the uptrend
                extremes_log.append(df['last_box_log'].iloc[i-1])
                columns_log.append(df['last_box_log'].iloc[i-1])
                # Record the first box of the downtrend
                columns_log.append(df['last_box_log'].iloc[i-1] - 1)

            else:
                # Nothing new
                df['trend'].iloc[i] = df['trend'].iloc[i-1]
                df['last_box'].iloc[i] =  df['last_box'].iloc[i-1]
                df['last_box_log'].iloc[i] = df['last_box_log'].iloc[i-1]
                df['column_height'].iloc[i] = df['column_height'].iloc[i-1]

        # If there is a downtrend
        elif df['trend'].iloc[i-1] == 'down':
            # Check for new box(es) to the downside
            if df['log_low'].iloc[i] <= (df['last_box_log'].iloc[i-1] - 1):
                df['trend'].iloc[i] = df['trend'].iloc[i-1]
                df['last_box_log'].iloc[i] = math.ceil(df['log_low'].iloc[i])  # 20250520 SE round up even if log is negative
                df['last_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['new_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['column_height'].iloc[i] = extremes_log[-1] - df['last_box_log'].iloc[i]

            # Check for reversal
            elif df['log_high'].iloc[i] >= df['last_box_log'].iloc[i-1] + reversal_threshold:
                df['trend'].iloc[i] = 'up'
                df['last_box_log'].iloc[i] = math.floor(df['log_high'].iloc[i])  # 20250520 SE round down even if log is negative
                df['last_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['reversal'].iloc[i] = True
                df['reversal_box'].iloc[i] = (1 + (box_percentage / 100)) ** (df['last_box_log'].iloc[i-1] + reversal_threshold)
                df['new_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['column_height'].iloc[i] = df['last_box_log'].iloc[i] - df['last_box_log'].iloc[i-1]
                # Record the bottom box of the downtrend
                extremes_log.append(df['last_box_log'].iloc[i-1])
                columns_log.append(df['last_box_log'].iloc[i-1])
                # Record the first box of the uptrend
                columns_log.append(df['last_box_log'].iloc[i-1] + 1)

            else:
                # Nothing new
                df['trend'].iloc[i] = df['trend'].iloc[i-1]
                df['last_box'].iloc[i] =  df['last_box'].iloc[i-1]
                df['last_box_log'].iloc[i] = df['last_box_log'].iloc[i-1]
                df['column_height'].iloc[i] = df['column_height'].iloc[i-1]

        # If no trend has been established yet at the beginning of the time series
        else:
            if df['log_high'].iloc[i] >= initial_box + reversal_threshold:
                df['trend'].iloc[i] = 'up'
                df['last_box_log'].iloc[i] = math.floor(df['log_high'].iloc[i])  # 20250520 SE round down even if log is negative
                df['last_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['reversal'].iloc[i] = True
                df['reversal_box'].iloc[i] = (1 + (box_percentage / 100)) ** (initial_box + reversal_threshold)
                df['new_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['column_height'].iloc[i] = df['last_box_log'].iloc[i] - initial_box
                columns_log.append(initial_box + 1)

            elif df['log_low'].iloc[i] <= initial_box - reversal_threshold:
                df['trend'].iloc[i] = 'down'
                df['last_box_log'].iloc[i] = math.ceil(df['log_low'].iloc[i])  # 20250520 SE round up even if log is negative
                df['reversal'].iloc[i] = True
                df['reversal_box'].iloc[i] = (1 + (box_percentage / 100)) ** (initial_box - reversal_threshold)
                df['new_box'].iloc[i] = (1 + (box_percentage / 100)) ** df['last_box_log'].iloc[i]
                df['column_height'].iloc[i] = initial_box - df['last_box_log'].iloc[i]
                columns_log.append(initial_box - 1)

            else:
                df['left_truncate'].iloc[i] = True
                df['last_box'].iloc[i] = (1 + (box_percentage / 100)) ** initial_box
                df['last_box_log'].iloc[i] = initial_box 

    # Append the final values to the extremes and columns
    extremes_log.append(df['last_box_log'].iloc[-1])
    columns_log.append(df['last_box_log'].iloc[-1])

    return df, extremes_log, columns_log

### Function to render a point and figure chart

In [4]:
 def generate_point_and_figure_chart(info_df):

    # Function to update a point and figure chart column with continuation
    def update_pf_chart_column(pf_points, chart_df, i):

        # Add one or more new points to an existing point and figure chart column
        col_head = chart_df.columns[-1]

        if pf_points['trend'].iloc[i] == 'up':
            # Uptrend: Add one or more X's
            char = 'X'
            # Allow for the possibility of a jump of more than one new box
            chart_df.loc[((chart_df['price_left'] >= pf_points['last_box_previous'].iloc[i] + 1) &
                        (chart_df['price_left'] <= pf_points['last_box'].iloc[i])
                        ), col_head] = char
            # See if the month has changed
            if pf_points['new_month'].iloc[i] == True:
                # Use the number of the month, except convert October through December to hex
                char = str(pf_points['month'].iloc[i])
                if pf_points['month'].iloc[i] == 10:
                    char = 'A'
                elif pf_points['month'].iloc[i] == 11:
                    char = 'B'
                elif pf_points['month'].iloc[i] == 12:
                    char = 'C'
                chart_df.loc[(chart_df['price_left'] == pf_points['last_box'].iloc[i]), col_head] = char

        else:
            # Downtrend: Add one or more O's
            char = 'O'
            # Allow for the possibility of a jump of more than one new box
            chart_df.loc[((chart_df['price_left'] <= pf_points['last_box_previous'].iloc[i] - 1) &
                        (chart_df['price_left'] >= pf_points['last_box'].iloc[i])
                        ), col_head] = char
            # See if the month has changed
            if pf_points['new_month'].iloc[i] == True:
                # Use the number of the month, except convert October through December to hex
                char = str(pf_points['month'].iloc[i])
                if pf_points['month'].iloc[i] == 10:
                    char = 'A'
                elif pf_points['month'].iloc[i] == 11:
                    char = 'B'
                elif pf_points['month'].iloc[i] == 12:
                    char = 'C'
                chart_df.loc[(chart_df['price_left'] == pf_points['last_box'].iloc[i]), col_head] = char

        return(chart_df)

    # End of function to update a point and figure chart column with continuation

    box_df = pd.DataFrame()   # an empty DataFrame

    # Size the chart height based on the highest and lowest points
    box_df['price_left'] =  pd.Series(np.arange(0, 102, box_size)).sort_values(ascending=False)

    # After the earlier processing, the first remaining row in info_df should always be a reversal
    # Add the first column to the point and figure chart
    i = 0

    new_col = info_df.index[i].strftime("%Y-%m-%d")
    # Fill the column with hyphens
    char = '-'
    box_df[new_col] = pd.Series([char] * len(box_df))
    col_head = new_col

    # Is the first column trend up or down?
    if info_df['trend'].iloc[i] == 'up':
        # Uptrend: put X's on the chart

        char = 'X'
        # Place at least 3 X's at the appropriate prices
        box_df.loc[((box_df['price_left'] >= info_df['last_box_previous'].iloc[i] + 1) &
                    (box_df['price_left'] <= info_df['last_box'].iloc[i])
                   ), col_head] = char
        # See if the month has changed
        if info_df['new_month'].iloc[i] == True:
            # Use the number of the month, except convert October through December to hex
            char = str(info_df['month'].iloc[i])
            if info_df['month'].iloc[i] == 10:
                char = 'A'
            elif info_df['month'].iloc[i] == 11:
                char = 'B'
            elif info_df['month'].iloc[i] == 12:
                char = 'C'
            box_df.loc[(box_df['price_left'] == info_df['last_box'].iloc[i]), col_head] = char

    else:
        # Downtrend: put O's on the chart

        char = 'O'
        # Place at least 3 O's at the appropriate prices
        box_df.loc[((box_df['price_left'] <= info_df['last_box_previous'].iloc[i] - 1) &
                    (box_df['price_left'] >= info_df['last_box'].iloc[i])
                   ), col_head] = char
        # See if the month has changed
        if info_df['new_month'].iloc[i] == True:
            # Use the number of the month, except convert October through December to hex
            char = str(info_df['month'].iloc[i])
            if info_df['month'].iloc[i] == 10:
                char = 'A'
            elif info_df['month'].iloc[i] == 11:
                char = 'B'
            elif info_df['month'].iloc[i] == 12:
                char = 'C'
            box_df.loc[(box_df['price_left'] == info_df['last_box'].iloc[i]), col_head] = char

    # Increment the event and reversal counters
    i += 1

    # Build out the point and figure chart by adding more columns
    for i in range(1, info_df.shape[0]):

        if info_df['reversal'].iloc[i] == False:
            # Continuation
            box_df = update_pf_chart_column(info_df, box_df, i)

        else:
            # Reversal
            # Add a new column to the point and figure chart
            new_col = info_df.index[i].strftime("%Y-%m-%d")
            # Fill it with hyphens
            char = '-'
            box_df[new_col] = pd.Series([char] * len(box_df))
            col_head = new_col

            # Is the new trend up or down?
            if info_df['trend'].iloc[i] == 'up':
                # Uptrend: put X's on the chart
                char = 'X'
                # Place at least 3 X's at the appropriate prices
                box_df.loc[((box_df['price_left'] >= info_df['last_box_previous'].iloc[i] + 1) &
                            (box_df['price_left'] <= info_df['last_box'].iloc[i])
                           ), col_head] = char
                # See if the month has changed
                if info_df['new_month'].iloc[i] == True:
                    # Use the number of the month, except convert October through December to hex
                    char = str(info_df['month'].iloc[i])
                    if info_df['month'].iloc[i] == 10:
                        char = 'A'
                    elif info_df['month'].iloc[i] == 11:
                        char = 'B'
                    elif info_df['month'].iloc[i] == 12:
                        char = 'C'
                    box_df.loc[(box_df['price_left'] == info_df['last_box'].iloc[i]), col_head] = char

            else:
                # Downtrend: put O's on the chart
                char = 'O'
                # Place at least 3 O's at the appropriate prices
                box_df.loc[((box_df['price_left'] <= info_df['last_box_previous'].iloc[i] - 1) &
                            (box_df['price_left'] >= info_df['last_box'].iloc[i])
                           ), col_head] = char
                # See if the month has changed
                if info_df['new_month'].iloc[i] == True:
                    # Use the number of the month, except convert October through December to hex
                    char = str(info_df['month'].iloc[i])
                    if info_df['month'].iloc[i] == 10:
                        char = 'A'
                    elif info_df['month'].iloc[i] == 11:
                        char = 'B'
                    elif info_df['month'].iloc[i] == 12:
                        char = 'C'
                    box_df.loc[(box_df['price_left'] == info_df['last_box'].iloc[i]), col_head] = char


    # Replicate the "y axis" price labels at the right edge of the dataframe
    box_df['price'] = box_df['price_left']

    return box_df

### Loop through the S&P 500 stocks and determine the daily trend for each

In [5]:
# S&P 500
symbols = [
		'MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A',
		'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOG',
		'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP',
		'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT',
		'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP',
		'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B',
		'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BSX', 'BMY',
		'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR',
		'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE',
		'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG',
		'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME',
		'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ',
		'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD',
		'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DVA', 'DAY', 'DECK', 'DE',
		'DELL', 'DAL', 'DVN', 'DXCM', 'FANG', 'DLR', 'DG', 'DLTR', 'D', 'DPZ',
		'DASH', 'DOV', 'DOW', 'DHI', 'DTE', 'DUK', 'DD', 'EMN', 'ETN', 'EBAY',
		'ECL', 'EIX', 'EW', 'EA', 'ELV', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM',
		'EQT', 'EFX', 'EQIX', 'EQR', 'ERIE', 'ESS', 'EL', 'EG', 'EVRG', 'ES',
		'EXC', 'EXE', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FICO', 'FAST',
		'FRT', 'FDX', 'FIS', 'FITB', 'FSLR', 'FE', 'FI', 'F', 'FTNT', 'FTV',
		'FOX', 'BEN', 'FCX', 'GRMN', 'IT', 'GE', 'GEHC', 'GEV', 'GEN', 'GNRC',
		'GD', 'GIS', 'GM', 'GPC', 'GILD', 'GPN', 'GL', 'GDDY', 'GS', 'HAL',
		'HIG', 'HAS', 'HCA', 'DOC', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX',
		'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUBB', 'HUM', 'HBAN', 'HII',
		'IBM', 'IEX', 'IDXX', 'ITW', 'INCY', 'IR', 'PODD', 'INTC', 'ICE', 'IFF',
		'IP', 'IPG', 'INTU', 'ISRG', 'IVZ', 'INVH', 'IQV', 'IRM', 'JBHT', 'JBL',
		'JKHY', 'J', 'JNJ', 'JCI', 'JPM', 'JNPR', 'K', 'KVUE', 'KDP', 'KEY',
		'KEYS', 'KMB', 'KIM', 'KMI', 'KKR', 'KLAC', 'KHC', 'KR', 'LHX', 'LH',
		'LRCX', 'LW', 'LVS', 'LDOS', 'LEN', 'LII', 'LLY', 'LIN', 'LYV', 'LKQ',
		'LMT', 'L', 'LOW', 'LULU', 'LYB', 'MTB', 'MPC', 'MKTX', 'MAR', 'MMC',
		'MLM', 'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'META',
		'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'MOH',
		'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ',
		'NTAP', 'NFLX', 'NEM', 'NWS', 'NEE', 'NKE', 'NI', 'NDSN', 'NSC', 'NTRS',
		'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL',
		'OMC', 'ON', 'OKE', 'ORCL', 'OTIS', 'PCAR', 'PKG', 'PLTR', 'PANW', 'PARA',
		'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PEP', 'PFE', 'PCG', 'PM', 'PSX',
		'PNW', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU',
		'PEG', 'PTC', 'PSA', 'PHM', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX',
		'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RVTY', 'ROK', 'ROL', 'ROP',
		'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SRE', 'NOW', 'SHW',
		'SPG', 'SWKS', 'SJM', 'SW', 'SNA', 'SOLV', 'SO', 'LUV', 'SWK', 'SBUX',
		'STT', 'STLD', 'STE', 'SYK', 'SMCI', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW',
		'TTWO', 'TPR', 'TRGP', 'TGT', 'TEL', 'TDY', 'TER', 'TSLA', 'TXN', 'TPL',
		'TXT', 'TMO', 'TJX', 'TKO', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC',
		'TYL', 'TSN', 'USB', 'UBER', 'UDR', 'ULTA', 'UNP', 'UAL', 'UPS', 'URI',
		'UNH', 'UHS', 'VLO', 'VTR', 'VLTO', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VTRS',
		'VICI', 'V', 'VST', 'VMC', 'WRB', 'GWW', 'WAB', 'WBA', 'WMT', 'DIS',
		'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC', 'WY', 'WSM',
		'WMB', 'WTW', 'WDAY', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZTS'

          ]

In [6]:
output_path_trend = FILEPATH + 'point_and_figure_daily_trend_sp500.csv'

for symbol in symbols:
    daily_trend_df = pd.DataFrame()   # an empty DataFrame
    read_ok = True
    
    # Retrieve daily ohlc data from openbb
    try:
        df = obb.equity.price.historical(
            symbol=symbol,
            start_date = start_date_string,
            end_date = end_date_string,
            provider="cboe"             # 20250514 openbb calls using yfinance are broken
        )
    except:
        read_ok = False
        daily_trend_df = pd.DataFrame({'trend': 'error reading data', 'date': None,
                                  'symbol': [symbol], 'box': None})
    
    if read_ok == True:
        # Convert the dates to datetime format
        df.index = pd.to_datetime(df.index)  
        
        # Calculate the average true range for the past 4 years
        df['true_range'] = (np.maximum(df['high'], df['close'].shift(1))
                                    / np.minimum(df['low'], df['close'].shift(1)))
        atr = np.average(df['true_range'].dropna())
    
        # Set box size
        box_percentage = round(100 * (atr - 1), 1)
    
        # Calculate point and figure history
        pct_df, extremes_log, columns_log = calc_point_and_figure(df)
    
        # Identify each day's trend
        pct_df['date'] = pct_df.index
        daily_trend_df = pct_df.loc[(pct_df['left_truncate'] == False)][['trend', 'date']]

        if len(daily_trend_df) > 0:
            daily_trend_df['symbol'] = symbol
            daily_trend_df['box'] = box_percentage
    
        else:
            daily_trend_df = pd.DataFrame({'trend': 'unidentified', 'date': None,
                                          'symbol': [symbol], 'box': box_percentage})

    daily_trend_df.to_csv(output_path_trend, mode='a', header=not os.path.exists(output_path_trend), index=False)

### Loop through the S&P Midcap 400 stocks and determine the daily trend for each

In [7]:
# Midcap 400
symbols = [
        'AA', 'AAL', 'AAON', 'ACHC', 'ACI', 'ACM', 'ADC', 'AFG', 'AGCO', 'AIT',
        'ALE', 'ALGM', 'ALK', 'ALLY', 'ALV', 'AM', 'AMED', 'AMG', 'AMH', 'AMKR',
        'AN', 'ANF', 'APPF', 'AR', 'ARMK', 'ARW', 'ASB', 'ASGN', 'ASH', 'ATI',
        'ATR', 'AVNT', 'AVT', 'AVTR', 'AXTA', 'AYI', 'BBWI', 'BC', 'BCO', 'BDC',
        'BHF', 'BILL', 'BIO', 'BJ', 'BKH', 'BLD', 'BLKB', 'BMRN', 'BRBR', 'BRKR',
        'BRX', 'BURL', 'BWXT', 'BYD', 'CACI', 'CADE', 'CAR', 'CART', 'CASY', 'CAVA',
        'CBSH', 'CBT', 'CCK', 'CDP', 'CELH', 'CFR', 'CG', 'CGNX', 'CHDN', 'CHE',
        'CHH', 'CHRD', 'CHWY', 'CHX', 'CIEN', 'CIVI', 'CLF', 'CLH', 'CMA', 'CMC',
        'CNH', 'CNM', 'CNO', 'CNX', 'CNXC', 'COHR', 'COKE', 'COLB', 'COLM', 'COTY',
        'CPRI', 'CR', 'CROX', 'CRS', 'CRUS', 'CSL', 'CUBE', 'CUZ', 'CVLT', 'CW',
        'CXT', 'CYTK', 'DAR', 'DBX', 'DCI', 'DINO', 'DKS', 'DLB', 'DOCS', 'DOCU',
        'DT', 'DTM', 'DUOL', 'EEFT', 'EGP', 'EHC', 'ELF', 'ELS', 'EME', 'ENS',
        'ENSG', 'ENTG', 'EPR', 'EQH', 'ESAB', 'ESNT', 'EVR', 'EWBC', 'EXEL', 'EXLS',
        'EXP', 'EXPO', 'FAF', 'FBIN', 'FCFS', 'FCN', 'FFIN', 'FHI', 'FHN', 'FIVE',
        'FIX', 'FLEX', 'FLG', 'FLO', 'FLR', 'FLS', 'FN', 'FNB', 'FND', 'FNF',
        'FOUR', 'FR', 'FYBR', 'G', 'GAP', 'GATX', 'GBCI', 'GEF', 'GGG', 'GHC',
        'GLPI', 'GME', 'GMED', 'GNTX', 'GPK', 'GT', 'GTLS', 'GWRE', 'GXO', 'H',
        'HAE', 'HALO', 'HGV', 'HIMS', 'HLI', 'HLNE', 'HOG', 'HOMB', 'HQY', 'HR',
        'HRB', 'HWC', 'HXL', 'IBKR', 'IBOC', 'IDA', 'ILMN', 'INGR', 'IPGP', 'IRDM',
        'IRT', 'ITT', 'JAZZ', 'JEF', 'JHG', 'JLL', 'KBH', 'KBR', 'KD', 'KEX',
        'KMPR', 'KNF', 'KNSL', 'KNX', 'KRC', 'KRG', 'LAD', 'LAMR', 'LANC', 'LEA',
        'LECO', 'LFUS', 'LITE', 'LIVN', 'LNTH', 'LNW', 'LOPE', 'LPX', 'LSCC ', 'LSTR',
        'M', 'MAN', 'MANH', 'MASI', 'MAT', 'MEDP', 'MIDD', 'MKSI', 'MLI', 'MMS',
        'MORN', 'MSA', 'MSM', 'MTDR', 'MTG', 'MTN', 'MTSI', 'MTZ', 'MUR', 'MUSA',
        'NBIX', 'NEU', 'NFG', 'NJR', 'NLY', 'NNN', 'NOV', 'NOVT', 'NSA', 'NSP',
        'NVST', 'NVT', 'NWE', 'NXST', 'NXT', 'NYT', 'OC', 'OGE', 'OGS', 'OHI',
        'OKTA', 'OLED', 'OLLI', 'OLN', 'ONB', 'ONTO', 'OPCH', 'ORA', 'ORI', 'OSK',
        'OVV', 'OZK', 'PAG', 'PB', 'PBF', 'PCH', 'PCTY', 'PEGA', 'PEN', 'PFGC',
        'PII', 'PK', 'PLNT', 'PNFP', 'POR', 'POST', 'POWI', 'PPC', 'PR', 'PRGO',
        'PRI', 'PSN', 'PSTG', 'PVH', 'QLYS', 'R', 'RBA', 'RBC', 'REXR', 'RGA',
        'RGEN', 'RGLD', 'RH', 'RLI', 'RMBS', 'RNR', 'ROIV', 'RPM', 'RRC', 'RRX',
        'RS', 'RYAN', 'RYN', 'SAIA', 'SAIC', 'SAM', 'SATS', 'SBRA', 'SCI', 'SEIC',
        'SF', 'SFM', 'SGI', 'SHC', 'SIGI', 'SKX', 'SLAB', 'SLGN', 'SLM', 'SMG',
        'SNV', 'SNX', 'SON', 'SR', 'SRPT', 'SSB', 'SSD', 'ST', 'STAG', 'STWD',
        'SWX', 'SYNA', 'TCBI', 'TEX', 'THC', 'THG', 'THO', 'TKR', 'TMHC', 'TNL',
        'TOL', 'TREX', 'TTC', 'TTEK', 'TXNM', 'TXRH', 'UA', 'UBSI', 'UFPI', 'UGI',
        'UMBF', 'UNM', 'USFD', 'UTHR', 'VAC', 'VAL', 'VC', 'VFC', 'VLY', 'VMI',
        'VNO', 'VNOM', 'VNT', 'VOYA', 'VVV', 'WAL', 'WBS', 'WCC', 'WEN', 'WEX',
        'WFRD', 'WH', 'WHR', 'WING', 'WLK', 'WMG', 'WMS', 'WPC', 'WSO', 'WTFC',
        'WTRG', 'WTS', 'WU', 'WWD', 'X', 'XPO', 'XRAY', 'YETI', 'ZI', 'ZION'
    ]

In [8]:
output_path_trend = FILEPATH + 'point_and_figure_daily_trend_midcap400.csv'

# Loop through symbols and determine the daily trend for each
for symbol in symbols:
    daily_trend_df = pd.DataFrame()   # an empty DataFrame
    read_ok = True
    
    # Retrieve daily ohlc data from openbb
    try:
        df = obb.equity.price.historical(
            symbol=symbol,
            start_date = start_date_string,
            end_date = end_date_string,
            provider="cboe"             # 20250514 openbb calls using yfinance are broken
        )
    except:
        read_ok = False
        daily_trend_df = pd.DataFrame({'trend': 'error reading data', 'date': None,
                                  'symbol': [symbol], 'box': None})
    
    if read_ok == True:
        # Convert the dates to datetime format
        df.index = pd.to_datetime(df.index)  
        
        # Calculate the average true range for the past 4 years
        df['true_range'] = (np.maximum(df['high'], df['close'].shift(1))
                                    / np.minimum(df['low'], df['close'].shift(1)))
        atr = np.average(df['true_range'].dropna())
    
        # Set box size
        box_percentage = round(100 * (atr - 1), 1)
    
        # Calculate point and figure history
        pct_df, extremes_log, columns_log = calc_point_and_figure(df)
    
        # Identify each day's trend
        pct_df['date'] = pct_df.index
        daily_trend_df = pct_df.loc[(pct_df['left_truncate'] == False)][['trend', 'date']]

        if len(daily_trend_df) > 0:
            daily_trend_df['symbol'] = symbol
            daily_trend_df['box'] = box_percentage
    
        else:
            daily_trend_df = pd.DataFrame({'trend': 'unidentified', 'date': None,
                                          'symbol': [symbol], 'box': box_percentage})

    daily_trend_df.to_csv(output_path_trend, mode='a', header=not os.path.exists(output_path_trend), index=False)

### Loop through the S&P Smallcap 600 stocks and determine the daily trend for each

In [9]:
# Small-cap 600
symbols = [
        'AAP', 'AAT', 'ABCB', 'ABG', 'ABM', 'ABR', 'ACA', 'ACAD', 'ACIW', 'ACLS',
        'ACT', 'ADEA', 'ADMA', 'ADNT', 'ADUS', 'AEIS', 'AEO', 'AESI', 'AGO', 'AGYS',
        'AHCO', 'AHH', 'AIN', 'AIR', 'AKR', 'AL', 'ALEX', 'ALG', 'ALGT', 'ALKS',
        'ALRM', 'AMN', 'AMPH', 'AMR', 'AMSF', 'AMTM', 'AMWD', 'ANDE', 'ANGI', 'ANIP',
        'AORT', 'AOSL', 'APAM', 'APLE', 'APOG', 'ARCB', 'ARI', 'ARLO', 'AROC', 'ARR',
        'ARWR', 'ASIX', 'ASO', 'ASTE', 'ASTH', 'ATEN', 'ATGE', 'AUB', 'AVA', 'AVAV',
        'AVNS', 'AWI', 'AWR', 'AX', 'AXL', 'AZTA', 'AZZ', 'BANC', 'BANF', 'BANR',
        'BCC', 'BCPC', 'BDN', 'BFH', 'BFS', 'BGC', 'BGS', 'BHE', 'BHLB', 'BJRI',
        'BKE', 'BKU', 'BL', 'BLFS', 'BLMN', 'BMI', 'BOH', 'BOOT', 'BOX', 'BRC',
        'BRKL', 'BSIG', 'BTU', 'BWA', 'BXMT', 'CABO', 'CAKE', 'CAL', 'CALM', 'CALX',
        'CARG', 'CARS', 'CASH', 'CATY', 'CBRL', 'CBU', 'CC', 'CCOI', 'CCS', 'CE',
        'CENT', 'CENX', 'CERT', 'CEVA', 'CFFN', 'CHCO', 'CHEF', 'CLB', 'CLSK', 'CNK',
        'CNMD', 'CNR', 'CNS', 'CNXN', 'COHU', 'COLL', 'CON', 'COOP', 'CORT', 'CPF',
        'CPK', 'CPRX', 'CRC', 'CRGY', 'CRI', 'CRK', 'CRSR', 'CRVL', 'CSGS', 'CSR',
        'CSWI', 'CTKB', 'CTRE', 'CTS', 'CUBI', 'CURB', 'CVBF', 'CVCO', 'CVI', 'CWEN',
        'CWK', 'CWT', 'CXM', 'CXW', 'DAN', 'DCOM', 'DEA', 'DEI', 'DFH', 'DFIN',
        'DGII', 'DIOD', 'DLX', 'DNOW', 'DOCN', 'DORM', 'DRH', 'DRQ', 'DV', 'DVAX',
        'DXC', 'DXPE', 'DY', 'EAT', 'ECG', 'ECPG', 'EFC', 'EGBN', 'EIG', 'ELME',
        'EMBC', 'ENOV', 'ENR', 'ENVA', 'EPAC', 'EPC', 'EPRT', 'ESE', 'ESI', 'ETD',
        'ETSY', 'EVTC', 'EXPI', 'EXTR', 'EYE', 'EZPW', 'FBK', 'FBNC', 'FBP', 'FBRT',
        'FCF', 'FCPT', 'FDP', 'FELE', 'FFBC', 'FHB', 'FIZZ', 'FL', 'FMC', 'FORM',
        'FOXF', 'FRPT', 'FSS', 'FTDR', 'FTRE', 'FUL', 'FULT', 'FUN', 'FWRD', 'GBX',
        'GDEN', 'GDYN', 'GEO', 'GES', 'GFF', 'GIII', 'GKOS', 'GMS', 'GNL', 'GNW',
        'GO', 'GOGO', 'GOLF', 'GPI', 'GRBK', 'GSHD', 'GTES', 'GTY', 'GVA', 'HAFC',
        'HASI', 'HAYW', 'HBI', 'HCC', 'HCI', 'HCSG', 'HELE', 'HFWA', 'HI', 'HIW',
        'HLIT', 'HLX', 'HMN', 'HNI', 'HOPE', 'HP', 'HRMY', 'HSII', 'HSTM', 'HTH',
        'HTLD', 'HTZ', 'HUBG', 'HWKN', 'HZO', 'IAC', 'IART', 'IBP', 'ICHR', 'ICUI',
        'IDCC', 'IIIN', 'IIPR', 'INDB', 'INN', 'INSP', 'INSW', 'INVA', 'IOSP', 'IPAR',
        'ITGR', 'ITRI', 'JACK', 'JBGS', 'JBLU', 'JBSS', 'JBT', 'JJSF', 'JOE', 'JXN',
        'KAI', 'KALU', 'KAR', 'KFY', 'KLG', 'KLIC', 'KMT', 'KN', 'KOP', 'KREF',
        'KRYS', 'KSS', 'KTB', 'KTOS', 'KW', 'KWR', 'LBRT', 'LCII', 'LEG', 'LGIH',
        'LGND', 'LKFN', 'LMAT', 'LNC', 'LNN', 'LPG', 'LQDT', 'LRN', 'LTC', 'LUMN',
        'LXP', 'LZB', 'MAC', 'MARA', 'MATW', 'MATX', 'MBC', 'MC', 'MCRI', 'MCW',
        'MCY', 'MD', 'MDU', 'MGEE', 'MGPI', 'MGY', 'MHO', 'MLAB', 'MLKN', 'MMI',
        'MMSI', 'MNRO', 'MODG', 'MOG.A', 'MP', 'MPW', 'MRCY', 'MRP', 'MRTN', 'MSEX',
        'MSGS', 'MTH', 'MTRN', 'MTUS', 'MTX', 'MWA', 'MXL', 'MYGN', 'MYRG', 'NABL',
        'NATL', 'NAVI', 'NBHC', 'NBTB', 'NEO', 'NEOG', 'NGVT', 'NHC', 'NMIH', 'NOG',
        'NPK', 'NPO', 'NSIT', 'NTCT', 'NVEE', 'NVRI', 'NWBI', 'NWL', 'NWN', 'NX',
        'NXRT', 'NYMT', 'OFG', 'OGN', 'OI', 'OII', 'OMCL', 'OMI', 'OSIS', 'OTTR',
        'OUT', 'OXM', 'PAHC', 'PARR', 'PAYO', 'PATK', 'PBH', 'PBI', 'PCRX', 'PDFS',
        'PEB', 'PECO', 'PENN', 'PFBC', 'PFS', 'PGNY', 'PHIN', 'PI', 'PINC', 'PIPR',
        'PJT', 'PLAB', 'PLAY', 'PLMR', 'PLUS', 'PLXS', 'PMT', 'POWL', 'PPBI', 'PRA',
        'PRAA', 'PRDO', 'PRG', 'PRGS', 'PRK', 'PRLB', 'PRVA', 'PSMT', 'PTEN', 'PTGX',
        'PUMP', 'PZZA', 'QDEL', 'QNST', 'QRVO', 'RAMP', 'RC', 'RCUS', 'RDN', 'RDNT',
        'RES', 'REX', 'REZI', 'RGR', 'RHI', 'RHP', 'RNST', 'ROCK', 'ROG', 'RUN',
        'RUSHA', 'RWT', 'RXO', 'SAFE', 'SABR', 'SAFT', 'SAH', 'SANM', 'SBCF', 'SBH',
        'SBSI', 'SCHL', 'SCL', 'SCSC', 'SCVL', 'SDGR', 'SEDG', 'SEE', 'SEM', 'SFBS',
        'SFNC', 'SGH', 'SHAK', 'SHEN', 'SHO', 'SHOO', 'SIG', 'SITC', 'SITM', 'SJW',
        'SKT', 'SKY', 'SKYW', 'SLG', 'SLP', 'SLVM', 'SM', 'SMP', 'SMPL', 'SMTC',
        'SNCY', 'SNDK', 'SNDR', 'SNEX', 'SONO', 'SPNT', 'SPSC', 'SPTN', 'SPXC', 'SSTK',
        'STAA', 'STBA', 'STC', 'STEL', 'STEP', 'STRA', 'STRL', 'SUPN', 'SXC', 'SXI',
        'SXT', 'TALO', 'TBBK', 'TDC', 'TDS', 'TDW', 'TFIN', 'TFX', 'TGI', 'TGNA',
        'TGTX', 'THRM', 'THRY', 'THS', 'TILE', 'TMDX', 'TMP', 'TNC', 'TNDM', 'TPH',
        'TR', 'TRIP', 'TRMK', 'TRN', 'TRNO', 'TRST', 'TRUP', 'TTGT', 'TTMI', 'TWI',
        'TWO', 'UCBI', 'UCTT', 'UE', 'UFCS', 'UFPT', 'UHT', 'UNF', 'UNFI', 'UNIT',
        'UPBD', 'URBN', 'USNA', 'USPH', 'UTL', 'UVV', 'VBTX', 'VCEL', 'VECO', 'VIAV',
        'VICR', 'VIR', 'VIRT', 'VRE', 'VRRM', 'VRTS', 'VSAT', 'VSCO', 'VSH', 'VSTS',
        'VTOL', 'VTLE', 'VVI', 'VYX', 'WABC', 'WAFD', 'WD', 'WDFC', 'WERN', 'WGO',
        'WHD', 'WKC', 'WLY', 'WOLF', 'WOR', 'WRLD', 'WS', 'WSC', 'WSFS', 'WSR',
        'WT', 'WWW', 'XHR', 'XNCR', 'XPEL', 'XRX', 'YELP', 'YOU', 'ZD', 'ZWS'
    ]

In [10]:
output_path_trend = FILEPATH + 'point_and_figure_daily_trend_smallcap600.csv'

# Loop through symbols and determine the daily trend for each
for symbol in symbols:
    daily_trend_df = pd.DataFrame()   # an empty DataFrame
    read_ok = True
    
    # Retrieve daily ohlc data from openbb
    try:
        df = obb.equity.price.historical(
            symbol=symbol,
            start_date = start_date_string,
            end_date = end_date_string,
            provider="cboe"             # 20250514 openbb calls using yfinance are broken
        )
    except:
        read_ok = False
        daily_trend_df = pd.DataFrame({'trend': 'error reading data', 'date': None,
                                  'symbol': [symbol], 'box': None})
    
    if read_ok == True:
        # Convert the dates to datetime format
        df.index = pd.to_datetime(df.index)  
        
        # Calculate the average true range for the past 4 years
        df['true_range'] = (np.maximum(df['high'], df['close'].shift(1))
                                    / np.minimum(df['low'], df['close'].shift(1)))
        atr = np.average(df['true_range'].dropna())
    
        # Set box size
        box_percentage = round(100 * (atr - 1), 1)
    
        # Calculate point and figure history
        pct_df, extremes_log, columns_log = calc_point_and_figure(df)
    
        # Identify each day's trend
        pct_df['date'] = pct_df.index
        daily_trend_df = pct_df.loc[(pct_df['left_truncate'] == False)][['trend', 'date']]

        if len(daily_trend_df) > 0:
            daily_trend_df['symbol'] = symbol
            daily_trend_df['box'] = box_percentage
    
        else:
            daily_trend_df = pd.DataFrame({'trend': 'unidentified', 'date': None,
                                          'symbol': [symbol], 'box': box_percentage})

    daily_trend_df.to_csv(output_path_trend, mode='a', header=not os.path.exists(output_path_trend), index=False)

## Calculate bullish percentage history

### Update point and figure parameters specific to the bullish percentage chart

In [11]:
box_size = 2.00
reversal_threshold = 3   # This must be a positive integer

### Read point and figure trends for each stock

In [12]:
# Read daily data for the 500 stocks in the S&P 500
FILENAME = 'point_and_figure_daily_trend_sp500.csv'
df = pd.read_csv(FILEPATH + FILENAME)

# Read daily data for the 400 stocks in the S&P Midcap 400
FILENAME = 'point_and_figure_daily_trend_midcap400.csv'
df2 = pd.concat([df, pd.read_csv(FILEPATH + FILENAME)])

# Read daily data for the 600 stocks in the S&P Small Cap 600
FILENAME = 'point_and_figure_daily_trend_smallcap600.csv'
df3 = pd.concat([df2, pd.read_csv(FILEPATH + FILENAME)])

### Count stocks in uptrends and downtrends each day

In [13]:
# Drop duplicate records
new_df = df3.drop_duplicates(keep='first')

In [14]:
bp_df = pd.crosstab(new_df['date'], new_df['trend'])

# Remove the first 70 days because many stocks do not have identified trends
bp_df = bp_df.iloc[70:]

In [15]:
# Data cleanup: remove any days with very few stocks
bp_df['total_stocks'] = bp_df['down'] + bp_df['up']
bp_df = bp_df.loc[bp_df['total_stocks'] > 1350]
bp_df.index = pd.to_datetime(bp_df.index)  # make index a date-time

In [16]:
# Calculate the bullish percentage
bp_df['bullish_pct'] = (100 * bp_df['up'] /
                        (bp_df['down'] + bp_df['up']))

### Calculate point and figure history of the bullish percentage

In [17]:
# Add more columns for determining point and figure
bp_df['trend'] = None
bp_df['last_box'] = None
bp_df['reversal'] = False
bp_df['reversal_box'] = None
bp_df['new_box'] = None
bp_df['column_height'] = 0

In [18]:
# Allow for a provisional period at the beginning before any trend is known
bp_df['left_truncate'] = False

In [19]:
# Initialize lists for point and figure chart of bullish percentage
extremes = []
columns = []

In [20]:
# Set a provisional initial box value
initial_box = box_size * round((bp_df['bullish_pct'] / box_size).iloc[0], 0).astype(int)
bp_df['left_truncate'].iloc[0] = True
bp_df['last_box'].iloc[0] = initial_box 

# Use this initial value as the starting point
extremes.append(initial_box)

In [21]:
# Loop through each day and update the point and figure chart of the bullish percentage
for i in range(1, bp_df.shape[0]):
    # If there is an uptrend
    # Check for new box(es) to the upside
    if bp_df['trend'].iloc[i-1] == 'up':
        if bp_df['bullish_pct'].iloc[i] >= (bp_df['last_box'].iloc[i-1] + box_size):
            bp_df['trend'].iloc[i] = bp_df['trend'].iloc[i-1]
            bp_df['last_box'].iloc[i] = box_size * math.floor((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['new_box'].iloc[i] = box_size * math.floor((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['column_height'].iloc[i] = int(bp_df['bullish_pct'].iloc[i] / box_size) - (extremes[-1] / box_size)

        # Check for reversal
        elif bp_df['bullish_pct'].iloc[i] <= bp_df['last_box'].iloc[i-1] - (box_size * reversal_threshold):
            bp_df['trend'].iloc[i] = 'down'
            bp_df['last_box'].iloc[i] = box_size * math.ceil((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['reversal'].iloc[i] = True
            bp_df['reversal_box'].iloc[i] = (bp_df['last_box'].iloc[i-1] - (box_size * reversal_threshold))
            bp_df['new_box'].iloc[i] = box_size * math.ceil((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['column_height'].iloc[i] = (bp_df['last_box'].iloc[i-1] / box_size) - math.ceil((bp_df['bullish_pct'].iloc[i]) / box_size)
            # Record the top box of the uptrend
            extremes.append(bp_df['last_box'].iloc[i-1])
            columns.append(bp_df['last_box'].iloc[i-1])
            # Record the first box of the downtrend
            columns.append(bp_df['last_box'].iloc[i-1] - box_size)

        else:
            # Nothing new
            bp_df['trend'].iloc[i] = bp_df['trend'].iloc[i-1]
            bp_df['last_box'].iloc[i] =  bp_df['last_box'].iloc[i-1]
            bp_df['column_height'].iloc[i] = bp_df['column_height'].iloc[i-1]


    # If there is a downtrend
    elif bp_df['trend'].iloc[i-1] == 'down':        
        # Check for new box(es) to the downside
        if bp_df['bullish_pct'].iloc[i] <= (bp_df['last_box'].iloc[i-1] - box_size):
            bp_df['trend'].iloc[i] = bp_df['trend'].iloc[i-1]
            bp_df['last_box'].iloc[i] = box_size * math.ceil((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['new_box'].iloc[i] = box_size * math.ceil((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['column_height'].iloc[i] = (extremes[-1] / box_size) - int(bp_df['bullish_pct'].iloc[i] / box_size)
            
        # Check for reversal
        elif bp_df['bullish_pct'].iloc[i] >= bp_df['last_box'].iloc[i-1] + (box_size * reversal_threshold):
            bp_df['trend'].iloc[i] = 'up'
            bp_df['last_box'].iloc[i] = box_size * math.floor((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['reversal'].iloc[i] = True
            bp_df['reversal_box'].iloc[i] = (bp_df['last_box'].iloc[i-1] + (box_size * reversal_threshold))
            bp_df['new_box'].iloc[i] = box_size * math.floor((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['column_height'].iloc[i] = math.floor((bp_df['bullish_pct'].iloc[i]) / box_size) - (bp_df['last_box'].iloc[i-1] / box_size)
            # Record the bottom box of the downtrend
            extremes.append(bp_df['last_box'].iloc[i-1])
            columns.append(bp_df['last_box'].iloc[i-1])
            # Record the first box of the uptrend
            columns.append(bp_df['last_box'].iloc[i-1] - box_size)

        else:
            # Nothing new
            bp_df['trend'].iloc[i] = bp_df['trend'].iloc[i-1]
            bp_df['last_box'].iloc[i] =  bp_df['last_box'].iloc[i-1]
            bp_df['column_height'].iloc[i] = bp_df['column_height'].iloc[i-1]

    # If no trend has been established yet at the beginning of the time series
    else:
        if bp_df['bullish_pct'].iloc[i] >= initial_box + (box_size * reversal_threshold):
            bp_df['trend'].iloc[i] = 'up'
            bp_df['last_box'].iloc[i] = box_size * math.floor((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['reversal'].iloc[i] = True
            bp_df['reversal_box'].iloc[i] = initial_box + (box_size * reversal_threshold)
            bp_df['new_box'].iloc[i] = box_size * math.floor((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['column_height'].iloc[i] = math.floor((bp_df['bullish_pct'].iloc[i]) / box_size) - (initial_box / box_size)
            columns.append(initial_box + box_size)

        elif bp_df['bullish_pct'].iloc[i] <= initial_box - (box_size * reversal_threshold):
            bp_df['trend'].iloc[i] = 'down'
            bp_df['last_box'].iloc[i] = box_size * math.ceil((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['reversal'].iloc[i] = True
            bp_df['reversal_box'].iloc[i] = initial_box - (box_size * reversal_threshold)
            bp_df['new_box'].iloc[i] = box_size * math.ceil((bp_df['bullish_pct'].iloc[i]) / box_size)
            bp_df['column_height'].iloc[i] = (initial_box / box_size) - math.ceil((bp_df['bullish_pct'].iloc[i]) / box_size)
            columns.append(initial_box - box_size)
            
        else:
            bp_df['left_truncate'].iloc[i] = True
            bp_df['last_box'].iloc[i] = initial_box 

In [22]:
# Append the final values to the extremes and columns
extremes.append(bp_df['last_box'].iloc[-1])
columns.append(bp_df['last_box'].iloc[-1])

In [23]:
# Get the previous bar's point and figure box
bp_df['last_box_previous'] = bp_df['last_box'].shift(1)

In [24]:
bp_df

trend,down,up,total_stocks,bullish_pct,trend,last_box,reversal,reversal_box,new_box,column_height,left_truncate,last_box_previous
date,,,,,,,,,,,,
2021-11-19,520,872,1392,62.643678,None,62.0,False,None,None,0,True,None
2021-11-22,534,861,1395,61.720430,None,62.0,False,None,None,0,True,62.0
2021-11-23,567,830,1397,59.413028,None,62.0,False,None,None,0,True,62.0
2021-11-24,578,820,1398,58.655222,None,62.0,False,None,None,0,True,62.0
2021-11-26,766,639,1405,45.480427,down,46.0,True,56.0,46.0,8,False,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-05,836,646,1482,43.589744,down,42.0,False,None,None,18,False,42.0
2025-08-06,833,649,1482,43.792173,down,42.0,False,None,None,18,False,42.0
2025-08-07,818,663,1481,44.767049,down,42.0,False,None,None,18,False,42.0


In [25]:
# Write daily counts to csv file
output_path_pctb = FILEPATH + 'pct_bullish_daily_counts.csv'
bp_df.to_csv(output_path_pctb, mode='a', header=not os.path.exists(output_path_pctb), index=True)

### Include SPY prices for analysis

In [26]:
start_date_string = bp_df.index[0]
end_date_string = bp_df.index[-1]

# Retrieve daily SPY prices
spy_df = obb.equity.price.historical(
    symbol='SPY',
    start_date = start_date_string,
    end_date = end_date_string,
    provider="cboe"             # 20250514 openbb calls using yfinance are broken
)

spy_df.index = pd.to_datetime(spy_df.index)  # make index a date-time

In [27]:
# Join SPY prices with bullish percentage counts
joined_df = pd.merge(bp_df, spy_df, on='date', how='inner')

In [28]:
# Create a dataframe of only the point and figure changes
# i.e., the bars that provided new information to the point and figure chart
info_df = joined_df.loc[(joined_df['new_box'].notnull()) & (joined_df['left_truncate'] == False)]

In [29]:
info_df['chg_to_next_box'] = np.log(info_df['close'].shift(-1)) - np.log(info_df['close'])

In [30]:
# Identify the month in order to use it in the point and figure chart
info_df['month'] = info_df.index.month
# Is it a different month from the last point in the point and figure chart?
info_df['new_month'] = (info_df['month'] != info_df['month'].shift(1))

In [31]:
info_df

,down,up,total_stocks,bullish_pct,trend,last_box,reversal,reversal_box,new_box,column_height,left_truncate,last_box_previous,open,high,low,close,volume,chg_to_next_box,month,new_month
date,,,,,,,,,,,,,,,,,,,,
2021-11-26,766,639,1405,45.480427,down,46.0,True,56.0,46.0,8,False,62.0,462.44,463.90,457.77,458.97,112537722,-0.007457,11,True
2021-11-30,964,452,1416,31.920904,down,32.0,False,None,32.0,16,False,46.0,462.00,464.03,455.30,455.56,142116364,-0.011169,11,False
2021-12-01,1034,384,1418,27.080395,down,28.0,False,None,28.0,18,False,32.0,461.68,464.67,450.29,450.50,128454291,0.006461,12,True
2021-12-03,1062,358,1420,25.211268,down,26.0,False,None,26.0,19,False,28.0,459.19,460.30,448.92,453.42,134827167,0.011774,12,False
2021-12-06,941,480,1421,33.779029,up,32.0,True,32.0,32.0,3,False,26.0,456.08,460.79,453.56,458.79,96662630,0.020474,12,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-25,509,974,1483,65.677680,down,66.0,False,None,66.0,6,False,68.0,635.12,637.58,634.84,637.10,55876018,-0.002892,7,False
2025-07-29,536,947,1483,63.857047,down,64.0,False,None,64.0,7,False,66.0,638.36,638.67,634.34,635.26,59588731,-0.001260,7,False
2025-07-30,592,891,1483,60.080917,down,62.0,False,None,62.0,8,False,64.0,636.00,637.68,631.54,634.46,77446461,-0.003758,7,False


In [32]:
# Write out the point and figure points to disk
info_output = info_df[['down', 'up', 'bullish_pct', 'trend', 'reversal', 'column_height', 
                       'last_box', 'last_box_previous', 'close', 'chg_to_next_box']]
info_output.to_csv(FILEPATH + "bullish_pct_point_and_figure_points.csv", index=True)

### Render a point and figure chart of the bullish percentage

In [33]:
box_df = generate_point_and_figure_chart(info_df)

In [34]:
box_df

,price_left,2021-11-26,2021-12-06,2021-12-20,2021-12-23,2022-01-07,2022-01-31,2022-02-18,2022-02-25,2022-03-07,...,2025-03-31,2025-04-08,2025-04-21,2025-04-23,2025-05-22,2025-06-11,2025-06-18,2025-07-01,2025-07-16,price
50,100.0,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,100.0
49,98.0,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,98.0
48,96.0,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,96.0
47,94.0,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,94.0
46,92.0,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,92.0
45,90.0,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,90.0
44,88.0,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,88.0
43,86.0,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,86.0
42,84.0,-,-,-,-,-,-,-,-,-,...,-,-,-,5,-,-,-,-,-,84.0
41,82.0,-,-,-,-,-,-,-,-,-,...,-,-,-,X,O,-,-,-,-,82.0


In [35]:
# Write chart to disk
box_df.iloc[:, 2:].to_csv(FILEPATH + "bullish_pct_pf_chart.csv", index=False)